In [46]:
# Libraries
import json
import jsonschema
import os
import numpy as np
import itertools
import shutil

## Create JSON data for the evacuation

### Color coding:
* <span style="color:lightblue">▉</span> Good but don't rush or consider
* <span style="color:lightgreen">▉</span> Not high prio but might be useful for debugging
* <span style="color:yellow">▉</span> Asap, where the testing will actually occur

### Params to test

#### <span style="color:lightblue">▉</span> Parameters
* Sim type : For extension
#### <span style="color:lightgreen">▉</span> Parameters
* Dims : Maybe just for different scenarios
* Map_seed if needed to make a map fixed (Don't really consider)
* evac_number
#### <span style="color:yellow">▉</span> Parameters
* Dynamic movement influence
* Aspiration influence and equation
* Ratio influence and equation
* fire spread
* lc/ld
* static influence

### Create validator function

In [47]:
schema = {
    "dim" : {
        "type" : "array",
        "maxItems" : 2,
        "items" : {
            "type" : "integer"
        }
    },
    "sim_type" : {
        "type" : "string",
        "items" : { "enum" : ["Flow" , "Total"]}
    },
    "param_seed" : {
        "type" : ["integer", "null"]
    },
    "setup" : {
        "type" : "object",
        "properties" : {
            "map_seed" : { "type" : ["number", "null"] },
            "initial_fire" : {
                "type" : ["array" , "null"],
                "maxItems" : 2,
                "items" : {
                    "type" : "integer"
                }
            },
            "initial_evac" : {
                "type" : ["array" , "null"],
                "items" : {
                    "type" : {
                        "type" : "object",
                        "properties" : {
                            "strategy" : {
                                "type" : "string",
                                "items" : {"enum" : ["Competititve" , "Cooperative"]}
                            },
                            "x" : {
                                "type" : "integer"
                            },
                            "y" : {
                                "type" : "integer"
                            },
                            "pr_c" : {
                                "type"  : "float"
                            }
                        }                        
                    }
                }
            },
            "evac_number" : {
                "type" : "integer"
            },
            "fire_spread" : {
                "type" : ["number","null"]
            },
            "lc" : {
                "type" : ["number","null"]
            },
            "ld" : {
                "type" : ["number","null"]
            },
        }
    },
    "fire" : {
        "type" : "object",
        "properties" : {
            "frontier" : {
                "type" : ["string" , "null"],
                "items" : {
                    "enum" : ["VecTree"]
                }
            },
            "movement" : {
                "type" : [
                    {
                        "type" : "object",
                        "properties" : {
                            "ClosestDistance" : {
                                "type" : ["number", "null"]
                            }
                        }
                    },
                ]
            },
            "aspiration" : {
                "type" : [
                    {
                        "type" : "object",
                        "properties" : {
                            "LogAspiration" :   {"type" : ["number", "null"]},
                            "RootAspiration" :   {"type" : ["number", "null"]} 
                        }
                    }
                ]
            },
            "ratio" : {
                "type" : [
                    {
                        "type" : "object",
                        "properties" : {
                            "Root" : {
                                "type" : ["number",  "null"]
                            },
                        }
                    },
                    {
                        "type" : "object",
                        "properties" : {
                            "Log" : {
                                "type" : ["number",  "null"]
                            }
                        }
                    },
                    {
                        "type" : "object",
                        "properties" : {
                            "Id" : {
                                "type" : ["number",  "null"]
                            }
                        }
                    },
                ]
   
            },
            "reward_game" : {
                "type" : [
                    {
                        "type" : "object",
                        "properties" : {
                            "InvLogRoot" : {
                                "type" : ["number",  "null"]
                            },
                            "RewardRoot" : {
                                "type" : ["number" , "null"]
                            }
                        }
                    },
                ]
   
            }
        }
        
    },
    "escape" : {
        "type" : "string",
        "items" : {"enum" : ["TimeTracker"]}
    },
    "death" : {
        "type" : "string",
        "items" : {"enum" : ["AnnounceInput"]}
    }, 
    "static_input" : {
        "type" : [
            {
                "type" : "object",
                "properties" : {
                    "ClosestToExit" : { "type" : ["number", "null"]}
                }
            }
        ]
    },
}

validator = jsonschema.Draft202012Validator(schema)
path = "../../inputs/tests/"
filename = "base_input.json"
with open(f"{path}{filename}") as f:
    js_data = json.load(f)
    print(validator.is_valid(js_data))
names = []
filedir = ""

True


In [48]:
# Default values
sim_type = "Total"
n = 51
dims = [n,n]
escape = "TimeTracker"
death = "AnnounceInput"
param_seed = 20

In [49]:
# Populations density
perc = [0.085]
evac_numbers = []
for p in perc:
    evac_numbers.append(int(n * n * p))

In [50]:
# Learning rates
# lc_over_ld = [0.5]
# lds = [0.4]
# lcldvals = []
# for rat in lc_over_ld:
#     for ld in lds:
#         lcldvals.push(
#             [rat * ld,ld]
#         )
lcldvals = [
    (0.3, 0.3)
]

In [51]:
# Static and Dynamic ratio
# dynamic_ratio = 1.
# soverd_ratios = [2.]
# soverd = []
# for rat in soverd_ratios:
#     soverd.push(
#         [rat * dynamic_ratio, dynamic_ratio]
#     )
d = 0.972430
ss = [1.48319]
static_vals = []
for v in ss:
    static_vals.append({
        "ClosestToExit" : v
    })

In [52]:
# Aspirations
asp = [0.01]


In [53]:
# Ratio
ratio = [0.6634011009326777]

In [54]:
# Reward game
reward_game = [0.913287201523695]

In [55]:
# FireSpread
f_s = np.around(np.arange(0.1,0.22,0.02),decimals=3)
names = tuple(f"f_s_test_val_{f}.json" for f in f_s)
filedir = "fire_spread"
print(f_s)

[0.1  0.12 0.14 0.16 0.18 0.2 ]


In [56]:
# Setup values

setups = []
for (n, fs, (lc,ld)) in itertools.product(
    evac_numbers,
    f_s,
    lcldvals
):
    s = {
        "evac_number" : n
    }
    if fs is not None:
        s["fire_spread"] = fs
    if lc is not None:
        s["lc"] = lc
    if ld is not None:
        s["ld"] = ld
    setups.append(s)

In [57]:
# Fire values
fires = []
for (a,r,rg) in itertools.product(
    asp,
    ratio,
    reward_game
):
    fire = {
        "movement" : {
            "ClosestDistance" : d 
        },
        "aspiration" : {
            "LogAspiration" : a 
            # "RootAspiration" : a 
        },
        "ratio" : {
            # "Root" : r 
            "Log" : r 
            # "Id" : r 
        },
        "reward_game" :  {
            # "InvLogRoot" : rg
            "RewardRoot" : rg
        }
    }
    fires.append(fire)

In [58]:
# Final
jsons = []
for (s, setup, fire) in itertools.product(
    static_vals,
    setups,
    fires,
):
    jsons.append({
        "dim" : dims,
        "param_seed" :param_seed,
        "sim_type" : sim_type,
        "setup" : setup,
        "fire" : fire,
        "escape" : escape,
        "death" : death,
        "static_input" : s
    })
for js in jsons:
    print(js)
    print(validator.is_valid(js))

{'dim': [51, 51], 'param_seed': 20, 'sim_type': 'Total', 'setup': {'evac_number': 221, 'fire_spread': 0.1, 'lc': 0.3, 'ld': 0.3}, 'fire': {'movement': {'ClosestDistance': 0.97243}, 'aspiration': {'LogAspiration': 1e-05}, 'ratio': {'Log': 0.6634011009326777}, 'reward_game': {'RewardRoot': 0.913287201523695}}, 'escape': 'TimeTracker', 'death': 'AnnounceInput', 'static_input': {'ClosestToExit': 1.48319}}
True
{'dim': [51, 51], 'param_seed': 20, 'sim_type': 'Total', 'setup': {'evac_number': 221, 'fire_spread': 0.12, 'lc': 0.3, 'ld': 0.3}, 'fire': {'movement': {'ClosestDistance': 0.97243}, 'aspiration': {'LogAspiration': 1e-05}, 'ratio': {'Log': 0.6634011009326777}, 'reward_game': {'RewardRoot': 0.913287201523695}}, 'escape': 'TimeTracker', 'death': 'AnnounceInput', 'static_input': {'ClosestToExit': 1.48319}}
True
{'dim': [51, 51], 'param_seed': 20, 'sim_type': 'Total', 'setup': {'evac_number': 221, 'fire_spread': 0.14, 'lc': 0.3, 'ld': 0.3}, 'fire': {'movement': {'ClosestDistance': 0.97243

In [59]:
## Clear folder

dir_to_write_to = os.path.abspath(f"{path}/{filedir}")
for filename in os.listdir(dir_to_write_to):
    file_path = os.path.join(dir_to_write_to, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [60]:
# Save data
assert len(jsons) == len(names)
dir_to_write_to = os.path.abspath(f"{path}/{filedir}")
json_obj = None
for (js,n) in zip(jsons, names):
    json_obj = json.dumps(js, indent=4)
    with open(f"{dir_to_write_to}\\{n}","w") as out:
        out.write(json_obj)
    json_obj = None
print(dir_to_write_to)

c:\Users\csdem\OneDrive\Desktop\Dissertation\evac-sim\sim\inputs\tests\fire_spread
